Hi Professor
I’m currently working in Jupyter Notebook to prototype and test each function of the project in isolation because I’m familiar with it.

Block 1 : User and Web interface | 
Block 2: Check URl validation

In [24]:
# This will be converted into py file
# Block:1 User and Web interface
def get_url():
    return input ("Website URL")

In [ ]:
url = get_url()
print("You entered:", url)
ok = False

# Block2: check url validation
if not valid_url(url):
    print ("Invalid format! Make sure it starts with http:// or https://")
else:
    ok, response = reachable_url(url) 
    if not ok: 
        print ("Sit not reachable. Check address or internet connection.")
    else:
         print(f"Site is reachable (status {response.status_code}).")



You entered: https://www.drd.pr.gov/
Site is reachable (status 200).


Block 3: Page Loader

In [57]:
# Download  playwright 
%pip install playwright
!playwright install


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [58]:
# Block 3: Page Loader (AI generated)
import nest_asyncio
nest_asyncio.apply()
from playwright.async_api import async_playwright

async def load_page_data_async(url: str) -> dict:
    """
    Async version: loads the page and returns a dict with:
      - html (str)
      - styles (list of dicts)
      - elements (list of tag names)
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page    = await browser.new_page()
        await page.goto(url, timeout=10000)

        html = await page.content()
        styles = await page.evaluate("""() => {
            const all = Array.from(document.querySelectorAll('*'));
            return all.map(el => {
                const cs = window.getComputedStyle(el);
                return {
                    tag: el.tagName,
                    id: el.id || null,
                    classes: el.className || null,
                    color: cs.color,
                    backgroundColor: cs.backgroundColor,
                    fontSize: cs.fontSize,
                    fontWeight: cs.fontWeight
                };
            });
        }""")
        await browser.close()

    return {
        "html": html,
        "styles": styles,
        "elements": [s["tag"] for s in styles]
    }

In [59]:
if ok:
    page_data = await load_page_data_async(url)
    print("HTML length:",      len(page_data["html"]))
    print("Elements found:",   len(page_data["elements"]))
    print("First style record:", page_data["styles"][0])
else:
    print("Skipping page load because the URL is invalid or unreachable.")

HTML length: 57035
Elements found: 660
First style record: {'tag': 'HTML', 'id': None, 'classes': 'w-mod-js w-mod-ix wf-montserrat-n9-active wf-montserrat-n1-active wf-montserrat-n7-active wf-montserrat-n5-active wf-montserrat-n4-active wf-montserrat-n2-active wf-montserrat-n3-active wf-montserrat-n6-active wf-montserrat-n8-active wf-montserrat-i4-active wf-montserrat-i9-active wf-montserrat-i6-active wf-montserrat-i8-active wf-montserrat-i2-active wf-montserrat-i7-active wf-montserrat-i5-active wf-montserrat-i3-active wf-montserrat-i1-active wf-active', 'color': 'rgb(0, 0, 0)', 'backgroundColor': 'rgba(0, 0, 0, 0)', 'fontSize': '16px', 'fontWeight': '400'}


Block 4: Accessibility Analyzer

In [ ]:
# Block 4: Accessibility Analyzer
# 1) Contrast 
%pip install wcag_contrast_ratio

In [83]:
# Contrast 
# Playwright gives us colors like "rgb(34, 34, 34)". The contrast library prefers hex (#222222), so add this function:
# 1) Contrast Checker 

import re
import wcag_contrast_ratio as contrast #this is the WCAG contrast library

def rgb_to_hex(rgb: str):
    """Convert rgb(r,g,b) to #rrggbb """ 
    rgb_match = re.match(r'rgba?\(\s*(\d+),\s*(\d+),\s*(\d+)', rgb)
    if not rgb_match: 
        return None
    r, g, b = map(int, rgb_match.groups())
    return f"#{r:02x}{g:02x}{b:02x}" #this builds the string for #rrggbb

def check_contrast(styles: list[dict], min_ratio: float = 4.5) -> list[tuple[str, float]]: #AI helped me 
    """
    Returns a list of (text_snippet, ratio) for every element 
    whose contrast ratio < min_ratio.
    """
    failures = []
    #we are getting the raw css and converting it to hex
    for item in styles:
        fg = rgb_to_hex(item["color"]) # foreground elemets text color 
        bg = rgb_to_hex(item["backgroundColor"]) # backgroud color 
        if not fg or not bg:
            continue
        # Take each pair of hex digits (positions 1–2, 3–4, 5–6), convert from hex to int (0–255), then divide by 255 → 0.0–1.0 (AI Generated)
        try:
            tup_fg = tuple(int(fg[i:i+2], 16)/255 for i in (1,3,5))
            tup_bg = tuple(int(bg[i:i+2], 16)/255 for i in (1,3,5))
            r = contrast.rgb(tup_fg, tup_bg)
        except Exception:
            continue
        if r < min_ratio:
            # take up to first 30 chars of visible text
            text = item.get("text","").strip().replace("\n"," ")
            # Truncate to 30 characters for readability, add ellipsis if needed
            snippet = text[:30] + ("…" if len(text)>30 else "")
            # If there’s no text at all, fall back to the tag name, e.g. "<BUTTON>"
            failures.append((snippet or f"<{item['tag']}>", round(r,2)))
    return failures


# 2) Typography Checker
# WCAG “large-scale text” thresholds (point sizes → CSS pixels at 96 dpi)
lg_text_px      = 18 * 96/72   # 18 pt normal → 24 px Based on WCAG scale of large-scale text 
lg_bold_text_px = 14 * 96/72   # 14 pt bold   → ~18.66 px Based on WCAG scale of large-scale text 
def px_to_int(px: str):
    if isinstance(px, str) and px.endswith("px"): #isintance checks whether a value “is an instance of” a given type. Also we want to be sure that px is actually a string—otherwise Python would throw an error.
        try:
            return int(px[:-2])
        except ValueError:
            return None
    return None
def is_large_text(size: int, weight: int): 
    bool
    if weight >= 700 and size >= lg_bold_text_px:
        return True
    if size >= lg_text_px:
        return True
    return False

def check_typography(styles: list[dict]): 
    list[dict]
    """
    Flags elements whose font-size/weight fall below WCAG minimum
    for “large-scale text.”  Returns a list of dicts:
      { tag, text_snippet, size, weight, is_large, needs_fix }
    """
    issues = []
    for item in styles:
        raw_size   = item.get("fontSize", "")
        raw_weight = item.get("fontWeight", "")

        size   = px_to_int(raw_size)            # px as integer
        try:
            weight = int(raw_weight)            # CSS weight is numeric
        except Exception:
            continue  # skip if we can’t parse weight

        if size is None:
            continue  # skip if size wasn’t in px

        large = is_large_text(size, weight)
        # WCAG does *not* require a minimum size for large text,
        # but for normal text it must be at least 18 pt (24 px).
        needs_fix = not large and size < lg_text_px

        if needs_fix:
            text = item.get("text", "").strip().replace("\n", " ")
            snippet = text[:30] + ("…" if len(text)>30 else "")
            issues.append({
                "tag":         item["tag"],
                "text":        snippet or f"<{item['tag']}>",
                "fontSize":    size,
                "fontWeight":  weight,
                "is_large":    large,
                "needs_fix":   needs_fix
            })

    return issues

In [87]:
# Running it 
# 1) Contrast failures
contrast_fails = check_contrast(page_data["styles"])
print("Low-contrast elements:", len(contrast_fails))
for text_snip, ratio in contrast_fails[:5]:
    print(f" • “{text_snip}” → ratio {ratio}:1")

# 2) Typography issues AI Helped
typo_issues = check_typography(page_data["styles"])
print("\nTypography issues:", len(typo_issues))
for issue in typo_issues[:5]:
    print(
        f" • “{issue['text']}” size={issue['fontSize']}px "
        f"weight={issue['fontWeight']} "
        f"is_large={issue['is_large']} needs_fix={issue['needs_fix']}"
    )
# Inspect the first issue’s keys
if typo_issues:
    print("Fields in each issue dict that need fix:", typo_issues[0].keys())
else:
    print("No typography issues found!")

Low-contrast elements: 512
 • “<HTML>” → ratio 1.0:1
 • “<HEAD>” → ratio 1.0:1
 • “<STYLE>” → ratio 1.0:1
 • “<META>” → ratio 1.0:1
 • “<TITLE>” → ratio 1.0:1

Typography issues: 617
 • “<HTML>” size=16px weight=400 is_large=False needs_fix=True
 • “<HEAD>” size=16px weight=400 is_large=False needs_fix=True
 • “<STYLE>” size=16px weight=400 is_large=False needs_fix=True
 • “<META>” size=16px weight=400 is_large=False needs_fix=True
 • “<TITLE>” size=16px weight=400 is_large=False needs_fix=True
Fields in each issue dict that need fix: dict_keys(['tag', 'text', 'fontSize', 'fontWeight', 'is_large', 'needs_fix'])
